In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

# Import Machine Learning Dependencies
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [3]:
#pip install sodapy

In [4]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [5]:
engine = create_engine("sqlite:///final_project")

In [6]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [7]:
results = client.get("xwdj-i9he", limit=1160000)

In [8]:
results_df = pd.DataFrame.from_records(results)

In [9]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_lat,sr_location_lat_long,sr_status_date,sr_location_street_name,sr_created_date,sr_location_street_number,sr_location_city,sr_location_y,sr_location_map_tile,sr_location_zip_code
466284,"PECOS ST & GILBERT ST, AUSTIN, TX",10,Phone,2019-08-02T01:00:54.000,3104130.50184992,Closed,Pothole Repair,TRAVIS,554X,-97.77425097,...,30.2989638,"{'latitude': '30.2989638', 'needs_recoding': F...",2019-08-02T01:00:54.000,PECOS ST & GILBERT ST,2019-02-02T11:14:28.000,NaN,AUSTIN,10081758.0010054,MG25,78703
409889,"2000 E WILLIAM CANNON DR, AUSTIN, TX 78744",2,Phone,2019-05-27T10:20:00.000,3108605.71011211,Closed,Austin Code - Request Code Officer,TRAVIS,674C,-97.76302445,...,30.18790525,"{'latitude': '30.18790525', 'needs_recoding': ...",2019-05-27T10:20:00.000,WILLIAM CANNON,2019-05-27T10:08:45.000,2000,AUSTIN,10041459.4372382,MH15,78744
1025767,"6609 HANSA LOOP, AUSTIN, TX",8,Phone,2015-01-15T14:47:54.000,3066079.63230887,Closed,Street Light Issue- Address,TRAVIS,641T,-97.8973724,...,30.19911853,"{'latitude': '30.19911853', 'needs_recoding': ...",2015-01-15T14:47:54.000,HANSA,2015-01-14T10:16:53.000,6609,AUSTIN,10044581.5807805,MA16,78739
899274,"12320 TOMANET TRL, AUSTIN, TX 78758",7,Phone,2015-12-08T15:12:26.000,3125430.49953578,Closed,Creek & Pond Vegetation Control,TRAVIS,466N,-97.70357582,...,30.41508134,"{'latitude': '30.41508134', 'needs_recoding': ...",2015-12-08T15:12:26.000,TOMANET,2015-12-06T08:36:24.000,12320,AUSTIN,10124496.9975579,ML35,78758
588498,"111 E CESAR CHAVEZ ST, AUSTIN, TX 78701",9,Phone,2018-04-30T10:29:10.000,3113959.99999781,Closed,Austin Code - Request Code Officer,TRAVIS,585W,-97.74405836,...,30.2630981,"{'latitude': '30.2630981', 'needs_recoding': F...",2018-04-30T10:29:10.000,CESAR CHAVEZ,2018-04-30T10:18:06.000,111,AUSTIN,10068938.0027979,MJ22,78701
147904,AVERY RANCH BLVD & N 183A SB TO AVERY RANCH RA...,6,Phone,2020-12-17T15:08:42.000,3092734.52268618,Closed,Traffic Signal - Maintenance,WILLIAMSON,403Q,-97.80545922,...,30.48647618,"{'latitude': '30.48647618', 'human_address': '...",2020-12-17T15:08:42.000,AVERY RANCH BLVD & N 183A SB TO AVERY RANCH,2020-12-17T14:46:47.000,NaN,AUSTIN,10149711.30822710,MF41,78613
574681,"7653 NORTHCROSS DR, AUSTIN, TX 78757",7,Phone,2018-12-09T01:02:39.000,3115902.94153378,Closed,Mowing Medians,TRAVIS,525K,-97.7354719,...,30.3534259,"{'latitude': '30.3534259', 'needs_recoding': F...",2018-12-09T01:02:39.000,NORTHCROSS,2018-05-29T19:58:12.000,7653,AUSTIN,10101843.5278212,MJ29,78757
408729,"JUNIPER ST & BRANCH ST, AUSTIN, TX 78702",1,Web,2019-08-01T14:58:09.000,3117964.49909908,Closed,Sign - New,TRAVIS,585T,-97.73116694,...,30.27065615,"{'latitude': '30.27065615', 'needs_recoding': ...",2019-08-01T14:58:09.000,JUNIPER ST & BRANCH ST,2019-05-29T10:38:03.000,NaN,AUSTIN,10071781.0031543,MJ22,78702
12976,"6917 PONDSDALE LN, AUSTIN, TX 78724",1,Phone,2021-09-01T13:55:29.000,3146297.81062005,Closed,ARR Dead Animal Collection,TRAVIS,557Y,-97.6405364,...,30.3012124,"{'latitude': '30.3012124', 'human_address': '{...",2021-09-01T13:55:29.000,PONDSDALE,2021-08-31T08:45:07.000,6917,AUSTIN,10083571.60201680,MP25,78724
676343,"S 1ST ST & W RIVERSIDE DR, AUSTIN, TX 78704",9,Phone,2017-08-21T18:28:50.000,3112339.99805867,Closed,Traffic Signal - Maintenance,TRAVIS,615A,-97.74928051,...,30.25975736,"{'latitude': '30.25975736', 'needs_recoding': ...",2017-08-21T18:28:50.000,S 1ST ST & W RIVERSIDE DR,2017-08-21T18:08:56.000,NaN,AUSTIN,10067685.0014488,MJ21,78704


In [10]:
len(results_df)

1157025

In [11]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [12]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [13]:
results_df['Request Month'] = pd.DatetimeIndex(results_df['Created Date']).month

In [14]:
results_df['Time of Day'] = pd.DatetimeIndex(results_df['Created Date']).hour

In [15]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [16]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [17]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [18]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [19]:
median = results_df['Total Seconds'].median(axis=0)

In [20]:
#median

In [21]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [22]:
conditions = [
    (results_df['Request Month'] == 1),
    (results_df['Request Month'] == 2),
    (results_df['Request Month'] == 3),
    (results_df['Request Month'] == 4),
    (results_df['Request Month'] == 5),
    (results_df['Request Month'] == 6),
    (results_df['Request Month'] == 7),
    (results_df['Request Month'] == 8),
    (results_df['Request Month'] == 9),
    (results_df['Request Month'] == 10),
    (results_df['Request Month'] == 11),
    (results_df['Request Month'] == 12)
]
values = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
results_df['Request Month'] = np.select(conditions, values)

In [23]:
#categorize the Hour variable
conditions = [
    (results_df['Time of Day'] >= 0) & (results_df['Time of Day'] < 4),
    (results_df['Time of Day'] >= 4) & (results_df['Time of Day'] < 8),
    (results_df['Time of Day'] >= 8) & (results_df['Time of Day'] < 12),
    (results_df['Time of Day'] >= 12) & (results_df['Time of Day'] < 16),
    (results_df['Time of Day'] >= 16) & (results_df['Time of Day'] < 20),
    (results_df['Time of Day'] >= 20) & (results_df['Time of Day'] <= 23)
    ]
values = ['Early Morning', 'Mid Morning', 'Late Morning', 'Afternoon', 'Evening', 'Night']
results_df['Time of Day'] = np.select(conditions, values)

In [24]:
timecheck = results_df['Time of Day'].value_counts()
timecheck

Late Morning     107091
Afternoon         96947
Evening           69938
Night             25225
Mid Morning       21088
Early Morning      6026
Name: Time of Day, dtype: int64

In [25]:
results_df.head()

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_zip_code,Created Date,Closed Date,Request Year,Request Month,Time of Day,Open Time,Open Time delta,Total Seconds,Response Time
300880,"6510 BERKMAN DR, AUSTIN, TX 78723",1,Phone,2020-01-02T18:13:51.000,3130569.45570825,Closed,Loud Commercial Music,TRAVIS,556K,-97.68991552,...,78723,2019-12-31 23:57:21,2020-01-02 18:13:51,2019,December,Night,1 days 18:16:30,1 days 18:16:30,152190.0,Slow Response
300881,"1800 E STASSNEY LN, AUSTIN, TX 78744",2,Spot311 Interface,2020-01-02T18:13:45.000,3109221.74865566,Closed,Loud Commercial Music,TRAVIS,644U,-97.76081753,...,78744,2019-12-31 23:44:51,2020-01-02 18:13:45,2019,December,Night,1 days 18:28:54,1 days 18:28:54,152934.0,Slow Response
300882,"400 E 7TH ST, AUSTIN, TX 78701",9,Web,2020-01-02T07:03:59.000,3115538.40203928,Closed,Street Light Issue- Address,TRAVIS,585T,-97.7389242,...,78701,2019-12-31 23:35:12,2020-01-02 07:03:59,2019,December,Night,1 days 07:28:47,1 days 07:28:47,113327.0,Slow Response
300883,"200 E 15TH ST, AUSTIN, TX 78701",1,Spot311 Interface,2020-01-02T09:06:02.000,3115838.49761388,Duplicate (closed),ARR Dead Animal Collection,TRAVIS,585P,-97.737741,...,78701,2019-12-31 23:17:46,2020-01-02 09:06:02,2019,December,Night,1 days 09:48:16,1 days 09:48:16,121696.0,Slow Response
300884,"200 SAN JACINTO BLVD, AUSTIN, TX 78701",9,Phone,2020-01-02T18:13:44.000,3114645.13713938,Closed,Loud Commercial Music,TRAVIS,585W,-97.74186188,...,78701,2019-12-31 23:03:52,2020-01-02 18:13:44,2019,December,Night,1 days 19:09:52,1 days 19:09:52,155392.0,Slow Response


In [26]:
#print(results_df['Total Seconds'].max())

In [27]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

In [28]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

In [29]:
#response = results_df['Response Time'].value_counts()
#response

In [30]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [31]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [32]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [33]:
#top_dept = results_df['Department'].value_counts()
#top_dept

In [34]:
#print(results_df.columns)

In [35]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [36]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [37]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [38]:
#training_df = training_df.drop(columns=[])

In [39]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [40]:
training_df = training_df.rename(columns={'Zip Code':'ZipCode'})

In [41]:
training_df.head(10)

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Total Seconds,Response Time,Department
300880,Phone,78723,2019,December,Night,152190.0,Slow Response,Austin Code
300881,Spot311 Interface,78744,2019,December,Night,152934.0,Slow Response,Austin Code
300882,Web,78701,2019,December,Night,113327.0,Slow Response,Public Works
300883,Spot311 Interface,78701,2019,December,Night,121696.0,Slow Response,Austin Resource Recovery
300884,Phone,78701,2019,December,Night,155392.0,Slow Response,Austin Code
300885,Phone,78745,2019,December,Night,18479378.0,Slow Response,Animal Services
300886,Spot311 Interface,78745,2019,December,Night,155819.0,Slow Response,Austin Code
300887,Phone,78746,2019,December,Night,1097446.0,Slow Response,Austin Resource Recovery
300888,Spot311 Interface,78745,2019,December,Night,117788.0,Slow Response,Animal Services
300889,Spot311 Interface,78660,2019,December,Night,162582.0,Slow Response,Animal Services


In [42]:
#read in census data from csv exports from data.census.gov 
mfi_data = "circle/MFI by Zipcode.csv"
pop_data = "circle/Population by Zipcode.csv"

mfi_df = pd.read_csv(mfi_data)
pop_df = pd.read_csv(pop_data)

In [43]:
#drop 0 index row
pop_df = pop_df.drop(0)

In [44]:
mfi_df = mfi_df.drop(0)

In [45]:
#regex to match just zip code 
pop_df['NAME'] = pop_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [46]:
mfi_df['NAME'] = mfi_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [47]:
pop_df = pop_df.rename(columns={'NAME':'ZipCode','B01003_001E':'TotalPopulation'})

In [48]:
pop_df = pop_df.drop(columns=['GEO_ID','B01003_001M'])

In [49]:
mfi_df = mfi_df.rename(columns={'NAME':'ZipCode','S1903_C03_015E':'MedianFamilyIncome'})

In [50]:
mfi_df = mfi_df.drop(columns=['GEO_ID'])

In [51]:
joined_df = mfi_df.join(pop_df.set_index('ZipCode'), on='ZipCode')

In [52]:
joined_df.head()

,ZipCode,MedianFamilyIncome,TotalPopulation
1,78610,88053,38525
2,78613,115556,81943
3,78617,63405,28930
4,78641,108589,72078
5,78652,103846,5374


In [53]:
training_df = training_df.join(joined_df.set_index('ZipCode'), on='ZipCode')

In [54]:
training_df = training_df.drop(columns = ["Total Seconds"])

In [55]:
training_df.head()

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Response Time,Department,MedianFamilyIncome,TotalPopulation
300880,Phone,78723,2019,December,Night,Slow Response,Austin Code,73162,35725
300881,Spot311 Interface,78744,2019,December,Night,Slow Response,Austin Code,52730,50123
300882,Web,78701,2019,December,Night,Slow Response,Public Works,175993,10848
300883,Spot311 Interface,78701,2019,December,Night,Slow Response,Austin Resource Recovery,175993,10848
300884,Phone,78701,2019,December,Night,Slow Response,Austin Code,175993,10848


In [56]:
#training_df.loc[training_df['MedianFamilyIncome'] == '-']

In [57]:
training_df.drop(training_df[training_df['MedianFamilyIncome'] == '-'].index, inplace = True)

In [58]:
#training_df['MedianFamilyIncome'].value_counts()

In [59]:
#training_df.dtypes

In [60]:
training_df['MedianFamilyIncome'] = pd.to_numeric(training_df['MedianFamilyIncome'])

In [61]:
eighty_mfi = 57260
onetwenty_mfi = 85891
conditions = [
    (training_df['MedianFamilyIncome'] < eighty_mfi),
    (training_df['MedianFamilyIncome'] >= eighty_mfi) & (training_df['MedianFamilyIncome'] <= onetwenty_mfi),
    (training_df['MedianFamilyIncome'] > onetwenty_mfi)
    ]
values = ['Below Median', 'Median', 'Above Median']
training_df['MedianFamilyIncome'] = np.select(conditions, values)

In [62]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(300880, 'Phone', '78723', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '35725'),
 (300881, 'Spot311 Interface', '78744', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Below Median', '50123'),
 (300882, 'Web', '78701', 2019, 'December', 'Night', 'Slow Response', 'Public Works', 'Above Median', '10848'),
 (300883, 'Spot311 Interface', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '10848'),
 (300884, 'Phone', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Above Median', '10848'),
 (300885, 'Phone', '78745', 2019, 'December', 'Night', 'Slow Response', 'Animal Services', 'Median', '61907'),
 (300886, 'Spot311 Interface', '78745', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '61907'),
 (300887, 'Phone', '78746', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '28608'),
 (300888, 'Spot311 Interface', '78745', 2019, 'De

Machine Learning Model

In [63]:
# Create our features
X = training_df.drop(columns = ["Response Time"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = training_df[["Response Time"]]

In [64]:
# Check the balance of our target values
y["Response Time"].value_counts()

Quick Response    162934
Slow Response     162914
Name: Response Time, dtype: int64

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [66]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train.values.ravel())

BalancedRandomForestClassifier(random_state=1)

In [67]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.7519801073377378

In [68]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[29412, 11227],
       [ 8972, 31851]], dtype=int64)

In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                      pre       rec       spe        f1       geo       iba       sup

Quick Response       0.77      0.72      0.78      0.74      0.75      0.56     40639
 Slow Response       0.74      0.78      0.72      0.76      0.75      0.57     40823

   avg / total       0.75      0.75      0.75      0.75      0.75      0.56     81462



In [70]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Department_Austin Code: (0.28294577773156254)
Department_Austin Resource Recovery: (0.07735372400038526)
Request Year: (0.05461155441458544)
Department_Austin Transportation: (0.0431116952770726)
Department_Animal Services: (0.034580991828502786)
Department_Public Works: (0.029532387743694665)
Report Method_Phone: (0.018533467483923413)
Department_Watershed Check: (0.01691195330495492)
Request Month_June: (0.016703479564884525)
Report Method_Spot311 Interface: (0.016666979077594398)
Time of Day_Afternoon: (0.015867422426028725)
Time of Day_Late Morning: (0.015540612020989749)
Time of Day_Night: (0.01492081933850692)
Request Month_May: (0.014603021335660056)
Request Month_July: (0.014360647678846537)
Time of Day_Evening: (0.013926293481877246)
Request Month_October: (0.013526741646902051)
Request Month_April: (0.01326752570964531)
Time of Day_Early Morning: (0.012848365980697829)
Request Month_November: (0.01250023670407349)
Request Month_August: (0.012451709319121141)
Request Month_Mar